<a href="https://colab.research.google.com/github/otviio/Alura_teste/blob/main/buscar_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai

In [4]:
import numpy as np
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

api_key = userdata.get('SECRET')
genai.configure(api_key=api_key)

In [5]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [6]:
title = "A proxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A proxima geração de IA para desenvolvedores e Google Workspace"
                "\n"
                "Artigo completo:\n"
                "Gemini API & Google AI Studio: Uma maneira acessivl de eplorar e criar prototipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.052033655, -0.043063007, -0.020352924, 0.02110293, 0.061900508, -0.0015466728, -0.021171594, -0.024289526, 0.055924788, 0.05950208, 0.015494063, 0.015773462, -0.04839876, -0.02099922, 0.0065578674, -0.018689297, 0.017631264, -0.005373566, -0.03540263, -0.0014743955, 0.013069284, 0.005633555, -0.035622954, -0.07877513, -0.015525065, 0.022251671, 0.012978647, -0.040037774, -0.04485081, 0.031124637, -0.05222439, 0.049091965, -0.03711382, 0.016216353, -0.027917435, -0.041640375, -0.01678068, -0.046540752, 0.0027180403, -0.00013002816, 0.005589845, -0.08434528, -0.019728774, 0.013748866, -0.0054053124, -0.025387274, 0.04034618, 0.049221974, 0.027400943, -0.05533829, 0.030974042, 0.018320316, 0.065107286, -0.044638462, 0.00092225516, -0.008913756, 0.031092668, -0.030371338, 0.023885624, 0.010372844, -0.0048749615, 0.025411248, -0.010142693, 0.0584531, 0.025972007, -0.058362927, -0.045341454, -0.0023785946, 0.012651317, 0.037094496, 0.0007899505, 0.0024998381, 0.06180054, -0.

In [7]:
DOCUMENT1 = {
    "Título": "A Dança das Estrelas",
    "Conteúdo": "O céu noturno, um palco infinito, onde estrelas cintilam em uma coreografia cósmica, hipnotizando a alma com sua beleza serena."

}

DOCUMENT2 = {
    "Título": "Sussurros do Vento",
    "Conteúdo": "Entre as folhas farfalhantes, o vento sussurra segredos da natureza, histórias antigas e melodias do tempo."
}

DOCUMENT3 = {
    "Título": "O Abraço da Chuva",
    "Conteúdo": "Gotas de chuva caem como lágrimas do céu, banhando a terra em um abraço refrescante que nutre a vida e acalma a alma."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [8]:
df = pd.DataFrame(documents)

df.columns = ["Titulo", "Conteudo"]

df

,Titulo,Conteudo
0,A Dança das Estrelas,"O céu noturno, um palco infinito, onde estrela..."
1,Sussurros do Vento,"Entre as folhas farfalhantes, o vento sussurra..."
2,O Abraço da Chuva,"Gotas de chuva caem como lágrimas do céu, banh..."


In [9]:
model = "models/embedding-001"

In [10]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [11]:
df["Embeddings"] = df.apply(lambda row:
                            embed_fn(row["Titulo"],
                            row["Conteudo"]),
                            axis=1)
df

,Titulo,Conteudo,Embeddings
0,A Dança das Estrelas,"O céu noturno, um palco infinito, onde estrela...","[0.01739332, -0.012797436, -0.04443075, 0.0181..."
1,Sussurros do Vento,"Entre as folhas farfalhantes, o vento sussurra...","[0.0030718837, -0.0079766335, -0.015682, 0.017..."
2,O Abraço da Chuva,"Gotas de chuva caem como lágrimas do céu, banh...","[0.0521636, 0.0027733115, -0.02812127, -0.0063..."


In [12]:
def gerar_e_buscar(consulta, base, modelo):
  embed_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embed_consulta["embedding"])

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

In [13]:
consulta = "O que cai?"

trecho = gerar_e_buscar(consulta, df, model)

print(trecho)

Gotas de chuva caem como lágrimas do céu, banhando a terra em um abraço refrescante que nutre a vida e acalma a alma.


In [15]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [18]:
prompt = f"Reescreva esse texto de forma descontraida, sem adcionar informações que não faça parte do texto.: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

O céu tá chorando, soltando umas lágrimas de chuva que caem igual uma chuveira gostosa. É como se a Terra estivesse tomando um banho refrescante, que dá vida pras plantinhas e deixa a gente com o coraçãozinho tranquilo.
